# EDA Capstone Project - Electricity Forecast
---

Information: <br>
The aim of this script is to examine a proper EDA on the dataset.
<br><br>
Expected Output:
 * Visualization of important features (stored in ../eda_plots)
 * Precise insights for data cleaning and data preparation

---

## Setup

In [1]:
# Main data packages. 
import numpy as np
import pandas as pd

# Getting data
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': 'white', 'grid.color': '.8'}
)
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'
cmaps_hex = ['#193251','#FF5A36','#696969', '#7589A2','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 300



---

## Start EDA

---

### load and inspect data

In [2]:
# load data

# Load environment variables from .env file
load_dotenv()

# Get database connection parameters from environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

# Create the database URL
db_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create an engine
engine = create_engine(db_url)

# Define your query
query = 'SELECT * FROM "03_gold"."name_of_the_table"'

# Execute the query and load the data into a pandas DataFrame
df_check = pd.read_sql(query, engine)

# Display the DataFrame
print(df_check.head())
# df = pd.read_csv('')

In [3]:
# inspect data
# df.head()

In [ ]:
# Check for dtypes and NAs
# df.info()

In [ ]:
# Check numeric features
# df.describe()

In [ ]:
# Check duplicates
# print(df.duplicated().sum())

# if df.duplicated().sum() > 0:
#     df = df.drop_duplicates()

In [ ]:
# Check missing values
# df.isnull().sum()

First insights:
 * No. of rows:
 * No. of columns:
 * Mismatching dtypes:
 * NA's:
 * Duplicated values: 

---


### check features

Suggestion: use Seaborn for plotting.

In [ ]:
# Template for visualization over time
fig, ax = plt.subplots()
sns.lineplot(x='time_variable', y='y_variable', data=df, ax=ax)
ax.set(title='Title')
ax.set_xlabel('Time_Name')
ax.set_ylabel('y_variable_name [unit]');

fig.savefig("../eda_plots/plot_name_that_makes_sense.png")

In [4]:
# Recognize linear and non-linear correlations and outliers
sns.pairplot(df)
plt.show()


In [5]:
# Check correlation
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
# Plot wind and solar 

# Bar chart for total production
plt.bar(['Solar', 'Wind', 'Total'], [total_solar, total_wind, total_production])
plt.ylabel('Production Quantity')
plt.title('Comparison of Energy Production')
plt.show()

# Time series plot
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['SolarProduction'], label='Solar')
plt.plot(df['Date'], df['WindProduction'], label='Wind')
plt.xlabel('Date')
plt.ylabel('Production Quantity')
plt.title('Time Trend of Energy Production')
plt.legend()
plt.show()


---
### check trend

Here we want to see the trend of the price over the year/years. Later on, we can substract this rolling mean to de-trend the data and focus more on seasonality (weekly/daily). 

Understand again, if this is necessary for all variables?

#### price

In [ ]:
# Plot moving average of different length (week, month, year)
ma = [7, 30, 365]


smooth_daily_data_df = daily_data_df \
    .reset_index() \
    .assign(date = lambda x: x['date'].transform(pd.to_datetime))

# Smooth and plot
fig, ax = plt.subplots(len(ma)+1, 1, constrained_layout=True, sharex=True)
plt.suptitle('Price Development (Daily) - Moving Average Smoothing', y=1.02);

for i, m in enumerate(ma):
    smooth_daily_data_df[f'price_smooth_ma_{m}'] = smooth_daily_data_df['price'].rolling(window=m,center=True).mean() #compute the rolling mean

    sns.lineplot(x='date', y='price', label='Price (Signal)', data=smooth_daily_data_df,  ax=ax[i])
    sns.lineplot(x='date', y=f'price_smooth_ma_{m}', label=f'Price smoothed:\n ma = {m} days', data=smooth_daily_data_df, color=NF_ORANGE, ax=ax[i])

    ax[i].legend(title='', loc='center left', bbox_to_anchor=(1, 0.5))
    ax[i].set(title='', ylabel=r'$^\circ$C');
        
sns.lineplot(x='date', y=f'price_smooth_ma_{m}', label=f'Price smoothed:\n ma = {m} days', data=smooth_daily_data_df, color=sns_c[1], ax=ax[i+1])
ax[i+1].legend(title='', loc='center left', bbox_to_anchor=(1, 0.5))
ax[i+1].set(title='', ylabel='$')

    
fig.savefig("../eda_plots/Price_MA_Smoothing.png")

In [ ]:
# Plot the Average daily price trend over time
# Set the 'start_date' as an index if it is not yet set
df.set_index('start_date', inplace=True)

# Resampling of the data to daily average values
daily_data = df['land[€/MWh]'].resample('D').mean()

# Visualize the resampled data
plt.figure(figsize=(10, 5))
plt.plot(daily_data.index, daily_data, marker='o', linestyle='-')
plt.title('Average daily price trend over time')
plt.xlabel('Date')
plt.ylabel('Average price  [€/MWh]')
plt.grid(True)
plt.show()

#### temperature

In [ ]:
# Plot moving average of different length (week, month, year)
ma = [7, 30, 365]


smooth_daily_data_df = daily_data_df \
    .reset_index() \
    .assign(date = lambda x: x['date'].transform(pd.to_datetime))

# Smooth and plot
fig, ax = plt.subplots(len(ma)+1, 1, figsize=(12, 9), constrained_layout=True, sharex=True)
plt.suptitle('Temperature (Daily) - Moving Average Smoothing', y=1.02);

for i, m in enumerate(ma):
    smooth_daily_data_df[f'temp_smooth_ma_{m}'] = smooth_daily_data_df['temperature'].rolling(window=m,center=True).mean() #compute the rolling mean

    sns.lineplot(x='date', y='temperature', label='Temperature (Signal)', data=smooth_daily_data_df,  ax=ax[i])
    sns.lineplot(x='date', y=f'temp_smooth_ma_{m}', label=f'Temperature smoothed:\n ma = {m} days', data=smooth_daily_data_df, color=NF_ORANGE, ax=ax[i])

    ax[i].legend(title='', loc='center left', bbox_to_anchor=(1, 0.5))
    ax[i].set(title='', ylabel=r'$^\circ$C');
        
sns.lineplot(x='date', y=f'temp_smooth_ma_{m}', label=f'Temperature smoothed:\n ma = {m} days', data=smooth_daily_data_df, color=sns_c[1], ax=ax[i+1])
ax[i+1].legend(title='', loc='center left', bbox_to_anchor=(1, 0.5))
ax[i+1].set(title='', ylabel=r'$^\circ$C');

    
fig.savefig("../eda_plots/Temp_MA_Smoothing.png")

### hours of sunshine

In [ ]:
# Visualization of the hours of sunshine
plt.subplot(311)
plt.plot(df['SunshineHours'], label='Sunshine Hours')
plt.xlabel('Date')
plt.ylabel('Hours')
plt.legend(loc='best')



In [ ]:
# Sunshine Hours and Solar Production Over Time

import matplotlib.dates as mdates

# Assuming df is your DataFrame and it contains columns 'Date', 'SunshineHours', and 'SolarProduction'
df['Date'] = pd.to_datetime(df['Date'])  

fig, ax1 = plt.subplots(figsize=(10, 6))

# Create the first Y-axis for the hours of sunshine
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Sunshine Hours', color=color)
ax1.plot(df['Date'], df['SunshineHours'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create the second Y-axis for solar production
ax2 = ax1.twinx()  # Create a second Y-axis that divides the first axis
color = 'tab:red'
ax2.set_ylabel('Solar Production', color=color)  # we already handled the x-label with ax1
ax2.plot(df['Date'], df['SolarProduction'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Formatting the X-axis for dates
ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
fig.autofmt_xdate()  # Improves the display of date labels (skewed)

plt.title('Sunshine Hours and Solar Production Over Time')
plt.show()

---

## Insights

The following steps need to be addressed in the (automatized) data cleaning and feature engineering /selection process:
 * Step 1:

 * Step 2:
 
 * Step 3:

---